# Importações Necessárias

In [0]:
import requests
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Extração dos Dados via API

In [0]:
url = "https://apisidra.ibge.gov.br/values/t/354/g/2/v/allxp/p/all/c12963/all?formato=json"
requisicao = requests.get(url)
dados = requisicao.json()

### Criação do Dataframe

In [0]:
df_spark = spark.createDataFrame(dados) 
display(df_spark)

D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N,MC,MN,NC,NN,V
"Brasil, Grande Região e UF (Código)","Brasil, Grande Região e UF",Variável (Código),Variável,Ano (Código),Ano,Tipo de doença (Código),Tipo de doença,Unidade de Medida (Código),Unidade de Medida,Nível Territorial (Código),Nível Territorial,Valor
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120931,Total,1020,Unidades,1,Brasil,2245
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120937,Dengue,1020,Unidades,1,Brasil,1547
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120930,Total geral de municípios,1020,Unidades,1,Brasil,5564
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120932,Diarréia,1020,Unidades,1,Brasil,1517
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120933,Leptospirose,1020,Unidades,1,Brasil,197
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120934,Verminoses,1020,Unidades,1,Brasil,1394
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120935,Cólera,1020,Unidades,1,Brasil,54
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120936,Difteria,1020,Unidades,1,Brasil,65
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120938,Tifo,1020,Unidades,1,Brasil,26


# Tratamento dos Dados

### Renomeando as Colunas

In [0]:
df_spark = df_spark.withColumnRenamed("D1C","cod_regiao") \
                    .withColumnRenamed("D1N", "regiao")\
                    .withColumnRenamed("D2C","cod_variavel") \
                    .withColumnRenamed("D2N", "variavel") \
                    .withColumnRenamed("D3C", "Ano-Código") \
                    .withColumnRenamed("D3N", "Ano") \
                    .withColumnRenamed("D4C","cod_doenca") \
                    .withColumnRenamed("D4N", "doenca") \
                    .withColumnRenamed("MC","cod_medida") \
                    .withColumnRenamed("MN", "medida") \
                    .withColumnRenamed("NC","cod_nivel_territorial") \
                    .withColumnRenamed("NN", "nivel territorial") \
                    .withColumnRenamed("V","valor")

# Limpando os dados
df_spark = df_spark.filter(df_spark.valor != "Valor")
display(df_spark)

cod_regiao,regiao,cod_variavel,variavel,Ano-Código,Ano,cod_doenca,doenca,cod_medida,medida,cod_nivel_territorial,nivel territorial,valor
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120931,Total,1020,Unidades,1,Brasil,2245
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120937,Dengue,1020,Unidades,1,Brasil,1547
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120930,Total geral de municípios,1020,Unidades,1,Brasil,5564
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120932,Diarréia,1020,Unidades,1,Brasil,1517
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120933,Leptospirose,1020,Unidades,1,Brasil,197
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120934,Verminoses,1020,Unidades,1,Brasil,1394
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120935,Cólera,1020,Unidades,1,Brasil,54
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120936,Difteria,1020,Unidades,1,Brasil,65
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120938,Tifo,1020,Unidades,1,Brasil,26
1,Brasil,2597,Municípios com ocorrência de doenças associadas ao saneamento básico,2008,2008,120939,Malária,1020,Unidades,1,Brasil,159


In [0]:
df_spark.printSchema()

root
 |-- cod_regiao: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- cod_variavel: string (nullable = true)
 |-- variavel: string (nullable = true)
 |-- Ano-Código: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- cod_doenca: string (nullable = true)
 |-- doenca: string (nullable = true)
 |-- cod_medida: string (nullable = true)
 |-- medida: string (nullable = true)
 |-- cod_nivel_territorial: string (nullable = true)
 |-- nivel territorial: string (nullable = true)
 |-- valor: string (nullable = true)



### Adequando o Schema das Colunas

In [0]:
df_spark = (df_spark.withColumn("cod_regiao", col("cod_regiao").cast(IntegerType()))
            .withColumn("cod_variavel", col("cod_variavel").cast(IntegerType()))
            .withColumn("cod_ano", col("Ano-Código").cast(IntegerType()))
            .withColumn("ano", col("Ano").cast(IntegerType()))
            .withColumn("cod_doenca", col("cod_doenca").cast(IntegerType()))
            .withColumn("cod_medida", col("cod_medida").cast(IntegerType()))
            .withColumn("cod_nivel_territorial", col("cod_nivel_territorial").cast(IntegerType()))
            .withColumn("valor", col("valor").cast(IntegerType()))
         )

# Verificar as Alterações no schema
df_spark.printSchema()

root
 |-- cod_regiao: integer (nullable = true)
 |-- regiao: string (nullable = true)
 |-- cod_variavel: integer (nullable = true)
 |-- variavel: string (nullable = true)
 |-- Ano-Código: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- cod_doenca: integer (nullable = true)
 |-- doenca: string (nullable = true)
 |-- cod_medida: integer (nullable = true)
 |-- medida: string (nullable = true)
 |-- cod_nivel_territorial: integer (nullable = true)
 |-- nivel territorial: string (nullable = true)
 |-- valor: integer (nullable = true)
 |-- cod_ano: integer (nullable = true)



# Armazenando o Dataframe no DBFS

In [0]:
df_spark.write \
    .option("compression", "gzip") \
    .mode("overwrite") \
    .format("json") \
    .save("/FileStore/tables/API-publica/json_gzip/")